# Importing Libraries

In [1]:
import plotly.offline
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
import plotly.graph_objects as go
import plotly.express as px
import matplotlib
import seaborn as sns

C:\Users\FArshad\AppData\Local\anaconda3\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning:

loaded more than 1 DLL from .libs:
C:\Users\FArshad\AppData\Local\anaconda3\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Users\FArshad\AppData\Local\anaconda3\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll



#  Loading Data

In [2]:
data = pd.read_excel(r"G:\3D_plotly_Resistivity_Map_Plot\Data.xlsx",header=None)
x_coord = data[1]
y_coord = data[2]

# Density Data


In [3]:
dens_data=[]
for i in data.columns[3:]:
    dens_data.append(data[i])

In [4]:
minimum = data[data.columns[3:]].values.flatten().min()
maximum = data[data.columns[3:]].values.flatten().max()

In [5]:
X,Y = np.unique(x_coord),np.unique(y_coord)
xi = np.linspace(min(X),max(X),len(X))
yi = np.linspace(min(Y),max(Y),len(Y))
xi,yi = np.meshgrid(xi,yi)
X1, Y1 = np.meshgrid(X,Y)

In [6]:
position = np.array([1,5,9,15,23,30,35,50,70,90,95,100,110])
name = data[0]

In [7]:
surface_data=[]
height = []
for j,i in enumerate(dens_data):
    Z = griddata((x_coord,y_coord),i,(X1,Y1),method='cubic')
    z_data = position[j]*np.ones((27,27))
    height.append(z_data)
    surface_data.append(Z)

In [8]:
def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0 / (pl_entries - 1)
    pl_colorscale = []
    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k * h)[:3]) * 255))
        pl_colorscale.append([k * h, 'rgb' + str((C[0], C[1], C[2]))])
    return pl_colorscale

In [9]:
sns.color_palette("tab10")

[(0.12156862745098039, 0.4666666666666667, 0.7058823529411765),
 (1.0, 0.4980392156862745, 0.054901960784313725),
 (0.17254901960784313, 0.6274509803921569, 0.17254901960784313),
 (0.8392156862745098, 0.15294117647058825, 0.1568627450980392),
 (0.5803921568627451, 0.403921568627451, 0.7411764705882353),
 (0.5490196078431373, 0.33725490196078434, 0.29411764705882354),
 (0.8901960784313725, 0.4666666666666667, 0.7607843137254902),
 (0.4980392156862745, 0.4980392156862745, 0.4980392156862745),
 (0.7372549019607844, 0.7411764705882353, 0.13333333333333333),
 (0.09019607843137255, 0.7450980392156863, 0.8117647058823529)]

In [10]:
seismic_cmap = matplotlib.cm.get_cmap('hsv')
colorscale = matplotlib_to_plotly(seismic_cmap, 255)


In [12]:
fig = go.Figure()
for i in range(len(surface_data)):
    if i==0:
        fig.add_surface(x=xi, y=yi, z=height[i], surfacecolor=surface_data[i],colorscale=colorscale,
                        showscale=True, showlegend=True,name=name[i])
        fig.data[0].colorbar.x=-0.1
    else:
        fig.add_surface(x=xi, y=yi, z=height[i], surfacecolor=surface_data[i],colorscale=colorscale,
                        showscale=False, showlegend=True,name=name[i])
fig.update_scenes(
            xaxis_title_text="Latitude",
            yaxis_title_text='Longitude',
            zaxis_title_text='Depth',
            zaxis_autorange="reversed"
        )
fig.update_layout(legend={"title":"Double click this to select only layer"})

fig.show()                       